In [2]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers.pooling import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
import os
import numpy as np
from keras.applications.imagenet_utils import preprocess_input 
from keras.utils import np_utils
import random
import pprint
import cPickle
# from scipy.misc import imread, imresize, imsave
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.engine.topology import Merge, merge
import matplotlib.pyplot as plt
from PIL import Image
from keras.utils.visualize_util import plot
from scipy.ndimage.filters import gaussian_filter as gauss
from keras.models import load_model
%pylab inline

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plot', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

def ycbcr2rgb(im):
#     xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float)
    rgb[:,:,[1,2]] -= 128
    return np.uint8(rgb.dot(xform.T))

In [7]:
random.seed(0)

X_train = []
Y_train = []


# def getRandSamples(c):
#     return random.sample(range(0,200), c)

counter = 0
image_count = 60
for directory in os.listdir('./mitData/'):
#     l = os.listdir('./mitData/' + directory)
#     image_indices  = getRandSamples(image_count)
    for (index,image) in enumerate(os.listdir('./mitData/' + directory)):        
#         image = l[0]
#         if(index in image_indices):
        img = load_img('./mitData/' + directory + '/' + image, target_size=(224,224))  # this is a PIL image
        x = img_to_array(img)
        x = rgb2ycbcr(x)
        Y_train.append(x[:,:,1:])
        X_train.append(x[:,:,:1])
        
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [15]:
XX_train = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3] + 2))
XX_train[:,:,:,0:] = X_train
XX_train.shape

(1175, 224, 224, 3)

In [11]:
def getModel():
    ''' 
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    for i in range(9):
        vgg_model.layers.pop()    
    
    bnl = BatchNormalization() (vgg_model.layers[-1].output)
    cnl = Convolution2D(256, 1, 1, border_mode='same', dim_ordering='tf') (bnl)
    usl = UpSampling2D(dim_ordering='tf') (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-5].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(128, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    usl = UpSampling2D(dim_ordering='tf') (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-9].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(64, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    usl = UpSampling2D(dim_ordering='tf') (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-12].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(3, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    bnl = BatchNormalization() (vgg_model.layers[-14].output)
    ml = merge([cnl, bnl], mode='sum')
    cnl = Convolution2D(3, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    cnl = Convolution2D(2, 3, 3, border_mode='same', dim_ordering='tf') (cnl)

    tl_model = Model( input=vgg_model.input, output=cnl )
    for i in range(1, 14):
        tl_model.layers[i].trainable = False

    return tl_model

In [27]:
tl_model = getModel()
# pprint.PrettyPrinter(indent=2).pprint(tl_model.get_config())
# tl_model.summary()
# opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
tl_model.compile(loss='mae', optimizer="adam", metrics=["mae"])

In [28]:
import datetime
from datetime import timedelta
import pickle

before = datetime.datetime.now()
print(before)
for i in range(1,11):
    history = tl_model.fit(XX_train,Y_train ,batch_size=50, nb_epoch=50, verbose=2, shuffle=True, validation_split = 0.1)
    name = './90percentModels/model' + str(i*50)
    pickle.dump(history.history,open('./history90percent/history' + str(i*50) + ".p", 'wb'))
    tl_model.save(name)
after = datetime.datetime.now()

print(after)
timeDone = after - before
print(timeDone)

2017-03-22 06:28:00.292252
Train on 1057 samples, validate on 118 samples
Epoch 1/50
50s - loss: 97.4974 - mean_absolute_error: 97.4974 - val_loss: 124.6579 - val_mean_absolute_error: 124.6579
Epoch 2/50
40s - loss: 37.1587 - mean_absolute_error: 37.1587 - val_loss: 34.9614 - val_mean_absolute_error: 34.9614
Epoch 3/50
40s - loss: 21.8844 - mean_absolute_error: 21.8844 - val_loss: 26.8257 - val_mean_absolute_error: 26.8258
Epoch 4/50
40s - loss: 18.4873 - mean_absolute_error: 18.4873 - val_loss: 21.7972 - val_mean_absolute_error: 21.7972
Epoch 5/50
40s - loss: 16.8004 - mean_absolute_error: 16.8004 - val_loss: 20.2880 - val_mean_absolute_error: 20.2880
Epoch 6/50
40s - loss: 15.8717 - mean_absolute_error: 15.8717 - val_loss: 20.0662 - val_mean_absolute_error: 20.0662
Epoch 7/50
40s - loss: 15.3469 - mean_absolute_error: 15.3469 - val_loss: 18.5447 - val_mean_absolute_error: 18.5447
Epoch 8/50
40s - loss: 14.5155 - mean_absolute_error: 14.5155 - val_loss: 18.7956 - val_mean_absolute_err

In [2]:
# tl_model1 = load_model('../rakshit/colorizer/models/200EpochsMIT')
# tl_model2 = load_model('../models/model_mae_200.md')
# tl_model3 = load_model('../rakshit/colorizer/models/200EpochsMIT')

In [4]:
random.seed(0)

X_test = []
Y_test = []
actual_image = []

counter = 0
for directory in os.listdir('./testData/'):
#     l = os.listdir('./mitData/' + directory)
    for (index,image) in enumerate(os.listdir('./testData/' + directory)):        
#         image = l[0]
        img = load_img('./testData/' + directory + '/' + image, target_size=(224,224))  # this is a PIL image
        x = img_to_array(img)
        actual_image.append(img)
        x = rgb2ycbcr(x)
        Y_test.append(x[:,:,1:])
        X_test.append(x[:,:,:1])
X_test = np.array(X_test)
X_test = np.array(X_test)

In [5]:
XX_test = np.zeros((X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3] + 2))
XX_test[:,:,:,0:] = X_test
XX_test.shape

(36, 224, 224, 3)

In [6]:
# model1_predictions = tl_model1.predict(XX_test)[3]
model2_predictions = tl_model2.predict(XX_test)
# model3_predictions = tl_model3.predict(XX_test)

In [17]:
# prediction1 = tl_model2.predict(XX_test[1:2,:,:,:])

# prediction1 = prediction1.reshape(224,224,2)
# p = np.append(XX_test[1,:,:,0:1], prediction1, axis=2)
# b = ycbcr2rgb(p)
# plt.imshow(b)

In [18]:
# median_filter(b)
# plt.imshow(b)

In [7]:
lower = 255
higher = 240
def check_filter(i,j, img):
    if  (img[i,j,0] > 240 and img[i,j,1] >  higher and img[i,j,2] <= lower) or \
        (img[i,j,1] > 240 and img[i,j,2] >  higher and img[i,j,0] <= lower) or \
        (img[i,j,0] > 240 and img[i,j,2] >  higher and img[i,j,1] <= lower) or \
        (img[i,j,0] > 240 and img[i,j,1] <= lower and img[i,j,2] <= lower) or \
        (img[i,j,1] > 240 and img[i,j,2] <= lower and img[i,j,0] <= lower) or \
        (img[i,j,2] > 240 and img[i,j,0] <= lower and img[i,j,1] <= lower):
            return True
    return False

def apply_filter(i,j, filter_size, img):
    row_start = (0 if i-filter_size < 0 else i-filter_size)
    row_end =  (len(img) if i+filter_size+1 > len(img) else i+filter_size+1)
    col_start = (0 if j-filter_size < 0 else j-filter_size)
    col_end = (len(img[0]) if j+filter_size+1 > len(img[0]) else j+filter_size+1)
    img[i,j, 0] = np.median(img[row_start : row_end, col_start : col_end, 0])
    img[i,j, 1] = np.median(img[row_start : row_end, col_start : col_end, 1])
    img[i,j, 2] = np.median(img[row_start : row_end, col_start : col_end, 2])
    
def median_filter(img):
    filter_size  = 2
    oldVal = -1
    while True:
        c = 0
        for i in range(0,224):
            for j in range(0,224):
                if(check_filter(i,j,b)):
                    c+=1
                    apply_filter(i,j,filter_size,b)
        if c == oldVal:
            break
        oldVal = c

In [1]:
# total_images = len(model2_predictions)
# pylab.rcParams['figure.figsize'] = (20, total_images*5)
# for i in range(1,total_images): #enumerate([1,3,5,14,18]):
# #     val +=1
#     plt.subplot(total_images,4,1+4*(i-1))
#     plt.imshow(actual_image[i])
    
# #     plt.subplot(total_images,4,3+4*(val-1))
# #     model1_predictions[i] = model1_predictions[i].reshape(224,224,2)
# #     p = np.append(XX_test[i,:,:,0:1], model1_predictions[i], axis=2)
# #     b = ycbcr2rgb(p)
# #     plt.imshow(b)

#     plt.subplot(total_images,4,3+4*(i-1))
#     model2_predictions[i] = model2_predictions[i]
#     p = np.append(XX_test[i,:,:,0:1], model2_predictions[i], axis=2)
#     b = ycbcr2rgb(p)
#     median_filter(b)
#     plt.imshow(b)
    
#     plt.subplot(total_images,4,4+4*(i-1))
#     model2_predictions[i] = model2_predictions[i]#.reshape(224,224,2)
#     p = np.append(XX_test[i,:,:,0:1], model2_predictions[i], axis=2)
#     b = ycbcr2rgb(p)
#     plt.imshow(b)
    
    
#     plt.subplot(total_images,4,2+4*(i-1))
# #     model3_predictions[i] = model3_predictions[i]#.reshape(224,224,2)
# #     p = np.append(XX_test[i,:,:,0:1], model3_predictions[i], axis=2)
# #     d = ycbcr2rgb(p)
# #     plt.imshow(d)
#     plt.imshow(XX_test[i,:,:,0],cmap='gray')
    
# plt.savefig('./output5.jpg')

In [9]:
plt.savefig('./output55.jpg')